### 네이버 쇼핑몰
- 거의 모든 객체가 동적으로 구성되어 있음
    - 디스플레이되고 있는 상품의 수가 많음
    - 한 화면에 진열이 불가능하므로 스크롤바를 움직여서 하단으로 내려오면 창이 확장되면서 상품이 진열되는 방식
- 네이버 스토어 측에서 무분별한 크롤링 방지 정책을 세워둠
    - 화면 진열 방식을 무한 크롤링으로 변경
    - 특정 횟수만큼 스크룰하고 수집

In [1]:
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys # 콘트롤되는 브라우저에 직접 키보드 키 신호를 프로그램적으로 전달하기 위해 사용
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)
from selenium.webdriver.chrome.service import Service

In [2]:
service = Service()
options = webdriver.ChromeOptions()
#크롬 드라이버의 경로 작성
# browser  = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
browser = webdriver.Chrome(service=service, options=options)

In [3]:
#웹사이트 열기
# site 요청 (site loading 종료까지 기다리는 시간이 필요)
browser.get('https://www.naver.com')
# 대기시간이 없는 경우 객체가 표현되지 않을 수 있음
browser.implicitly_wait(10)

In [4]:
# 스토어는 창 크기에 따라 표현되어지는 form이 달라짐 : 전체화면으로 놓고 작업
# 창크기가 작으면 mobile mode로 바뀌는 경우도 있음(naver)
browser.maximize_window()

In [5]:
#스토어 메뉴 클릭(쇼핑 메뉴로 자동으로 넘어감)
#shortcutArea > ul > li:nth-child(4) > a
#shortcutArea > ul > li:nth-child(4) > a > span.service_icon.type_shopping
browser.find_element(By.CSS_SELECTOR, "#shortcutArea > ul > li:nth-child(4) > a")
browser.find_element(By.CSS_SELECTOR, "#shortcutArea > ul > li:nth-child(4) > a").click()

time.sleep(2)

In [7]:
#창이 2개 이상 열렸을대 핸들 확인
# 브라우저의 활성창이 쇼핑창이어도 프로그램내에서는 첫번째 열었던 창의 참조만 연결하고 있음
print(browser.window_handles)

['132F224C061B216B1932747584C2C50A', 'CA31F6557D346A55AFD98173002CA5C4']


In [8]:
# 스토어 창으로 핸들 전환
browser.switch_to.window(browser.window_handles[1])

In [9]:
# browser.close()

In [10]:
# 검색어 입력 창 태그 확인 (form > div)
# <div class="_gnbSearch_inner_INhOz">

#  검색창의 div 태그 내에 검색어 입력 태그(input tag)
sel = "._gnbSearch_inner_INhOz input"

# 검색어 입력창 찾아서 검색어 입력 및 enter키 전송
search = browser.find_element(By.CSS_SELECTOR, sel)
# input 태그에 검색어 전송(send_keys(전달할 키 값))
search.send_keys("고구마")
# Keys : dict로 구성, 키보드의 모든 키값을 저장하고 있음
# Key를 지칭할 수 있는 상수로 저장되어 있음
search.send_keys(Keys.ENTER)

In [11]:
# 스크롤이 하단으로 내려가면 스크립트가 실행되면서 새로 상품이 전시되는 구조
# 스크롤 전 높이 확인(자바 스크립트 명령어 실행)
# "window.scrollY" # y 스크롤바의 현재 위치값을 저장

# "return window.scrollY" # 자바스크림트 코드 실행(웹 드라이버 객체 통해서 실행 : exexute_script())
before_h = browser.execute_script("return window.scrollY")
before_h
# y 스크롤 바 변경 후에 before_h와 변경된 y 스크롤 위치를 비교해서 변화가 없으면 중지하는 방식

0

In [12]:
#스크롤(네이버가 크롤링을 방지하는 정책을 사용하고 있고 무한스크롤 되어도 위쪽 item이 없어지므로 적당히 스크롤한 후 진행)
# 더이상 스크롤 되지 않도록 무한 스크롤 해야 하나, 정책상 크롤링 거부될 수 있으므로 20번정도 스크롤 후 정보 추출
for i in range(0, 20):
    # 현재 화면의 가장 아래로 스크롤 진행(스크롤 바 움직이는 대신 body객체에게 end키를 보내서 화면 가장 아래로 이동하도록)
    browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
    time.sleep(1)

    after_h = browser.execute_script("return window.scrollY")
    print(after_h)
    if (after_h == before_h):
        break
    else :
        before_h = after_h
        

4935.2001953125
4935.2001953125


In [13]:
# # 무한스크롤  코드
# interval = 2
# prev_height = browser.execute_script("return document.body.scrollHeight")

# while True:
#     #화면 가장 아래로 스크롤 내리기
#     browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#     time.sleep(interval)

#     curr_height = browser.execute_script("return document.body.scrollHeight")
#     if curr_height == prev_height:
#         break
    
#     prev_height = curr_height

### 상품 정보 추출
- 전체 상품 정보 div 태그 추출 후
- 내부 li 태그 추출

In [14]:
# 전체 상품
# <div id="composite-card-list" class="compositeCardList_composite_card_list__55sy4">
items = browser.find_element(By.ID, "composite-card-list")
lis = items.find_elements(By.TAG_NAME, "li")
lis

[<selenium.webdriver.remote.webelement.WebElement (session="19b8be79e3a4d0887d90da3320c7ac90", element="f.CA31F6557D346A55AFD98173002CA5C4.d.677937984D7829F9A29CF83FF8EA3C31.e.529")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19b8be79e3a4d0887d90da3320c7ac90", element="f.CA31F6557D346A55AFD98173002CA5C4.d.677937984D7829F9A29CF83FF8EA3C31.e.530")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19b8be79e3a4d0887d90da3320c7ac90", element="f.CA31F6557D346A55AFD98173002CA5C4.d.677937984D7829F9A29CF83FF8EA3C31.e.531")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19b8be79e3a4d0887d90da3320c7ac90", element="f.CA31F6557D346A55AFD98173002CA5C4.d.677937984D7829F9A29CF83FF8EA3C31.e.532")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19b8be79e3a4d0887d90da3320c7ac90", element="f.CA31F6557D346A55AFD98173002CA5C4.d.677937984D7829F9A29CF83FF8EA3C31.e.533")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19b8be79e3a4d0887d90

In [15]:
# 한개 상품의 상품명
#<strong class="basicProductCardInformation_title__Bc_Ng">국내산 24년 햇고구마 꿀고구마10kg 꿀밤 고구마</strong><strong class="basicProductCardInformation_title__Bc_Ng">국내산 24년 햇고구마 꿀고구마10kg 꿀밤 고구마</strong>
# lis[0].text
# selenium 객체 내 html 태그 확인
lis[0].get_attribute("innerHTML") # 객체 내 태그 문자열로 반환
lis[0].find_element(By.CSS_SELECTOR, ".basicProductCardInformation_title__Bc_Ng").text

'밤고구마 세척 타박한 여주밤고구마 2kg5kg'

In [16]:
# 상품 가격
#<span class="priceTag_inner_price__TctbK">33,800<span class="priceTag_unit__IXyyy">원</span></span>
lis[0].find_element(By.CSS_SELECTOR, ".priceTag_inner_price__TctbK").text.split("\n")[0].replace(",", "")

'20000'

In [17]:
# 상품링크
# 해당 상품에 a태그는 1개 이므로 태그로 추출
lis[0].find_element(By.TAG_NAME, "a").get_attribute("href")

'https://adcr.naver.com/adcr?x=heiDQDpF3ohpcYp/Y6H5E////w==kSZppPOmW1l4VDBxcNMtoLDkt4TSmGYnJTDp2+jnn+VC5K7oWk7EeFOebgPZUy3Rs2EC7X2fN+KfV/AmSxOVf5D21mIjEXnymTBav/+Yp/DfjMvI1uRtBA1SQ+6X1UHVzxoRc5BSa41pPLvs7F5FEZ0qlMUpsU44Hd7ltlbMYWVKahfTP67i0XD3cKr3hTYzDboxuUKQAFMhOW3VQGNxtBU1W6BJ0VQpl8/ABruQL0S9S3jGxxfjqz/+uuk4eqhJ7lIoP19Xy0Dbe6BLSuygN//XO6feS3kfRrKEQyHku/secR4ykqt6aSgE3VQE8qh6WyQvIKynMnidPa3xXjMTCCWnEGVH8Rw9A48vH+ikgqE1d8tc1R3rZAu7BOeFFraMrIkZKMc7lZmjYPUBmCjXP5H9LEYeL8Y1b8owy6FLMjpyZe4BaO/TkVpwYAZL9ftzN5ZLptsR8O2dHnUthb0IxJXKhDdQMVNrdZHWQjwVYhMlslaxBC8mDBeILhov6UwEavB3Rld27S2atRLQzttWMs1I62AGerbDSdBEgFNNPuYQJDYL5Irmkt18UiTkfvqPJL4bPWspzfx5p7+T90wKrFC3qsropU0UqMa1f5l7Clf2CbrmAwxVkev2ouMlve0D8IzOjcTifdZ7WlSivCGoecHOR7e7uRehTrR+sEG5Ivu6sRMHdYDuWUaBEAo410qtYiWiRm+wy2HLkJ33GhCk5wqrkjJVKHFqlrRWkGY6t6eOTFfw893mXUu8Tr9ExnHLDjQMGCXCXBnLlKahK+ZQKgt7HgT6yrtr6+6pYe6As0ZTckFVUtLTrXeqCSxUcbl8yvZW3e0hpcnGDTyU+41P2cEvtqe2GNGeEnuWuIaE7TIrsQvfIRDs19GzAAMIfo+v8ovOjEAHJY5I4Yf7Rt6/Zug==#channelProductNo=48

In [ ]:
# 20번 스크롤해서 찾은 모든 상품에 대한 상품명, 가격, 링크 추출(출력)
for item in lis:
    try:
        name = item.find_element(By.CSS_SELECTOR,'.basicProductCardInformation_title__Bc_Ng').text    
        price = item.find_element(By.CSS_SELECTOR,'.priceTag_inner_price__TctbK').text.split('\n')[0].replace(',','')
        link = item.find_element(By.TAG_NAME,'a').get_attribute('href')
    except:
        name = '판매중단'
        price = '판매중단'
        link = '판매중단'
    print(name, price, link)
    break


밤고구마 세척 타박한 여주밤고구마 2kg5kg 20000 https://adcr.naver.com/adcr?x=heiDQDpF3ohpcYp/Y6H5E////w==kSZppPOmW1l4VDBxcNMtoLDkt4TSmGYnJTDp2+jnn+VC5K7oWk7EeFOebgPZUy3Rs2EC7X2fN+KfV/AmSxOVf5D21mIjEXnymTBav/+Yp/DfjMvI1uRtBA1SQ+6X1UHVzxoRc5BSa41pPLvs7F5FEZ0qlMUpsU44Hd7ltlbMYWVKahfTP67i0XD3cKr3hTYzDboxuUKQAFMhOW3VQGNxtBU1W6BJ0VQpl8/ABruQL0S9S3jGxxfjqz/+uuk4eqhJ7lIoP19Xy0Dbe6BLSuygN//XO6feS3kfRrKEQyHku/secR4ykqt6aSgE3VQE8qh6WyQvIKynMnidPa3xXjMTCCWnEGVH8Rw9A48vH+ikgqE1d8tc1R3rZAu7BOeFFraMrIkZKMc7lZmjYPUBmCjXP5H9LEYeL8Y1b8owy6FLMjpyZe4BaO/TkVpwYAZL9ftzN5ZLptsR8O2dHnUthb0IxJXKhDdQMVNrdZHWQjwVYhMlslaxBC8mDBeILhov6UwEavB3Rld27S2atRLQzttWMs1I62AGerbDSdBEgFNNPuYQJDYL5Irmkt18UiTkfvqPJL4bPWspzfx5p7+T90wKrFC3qsropU0UqMa1f5l7Clf2CbrmAwxVkev2ouMlve0D8IzOjcTifdZ7WlSivCGoecHOR7e7uRehTrR+sEG5Ivu6sRMHdYDuWUaBEAo410qtYiWiRm+wy2HLkJ33GhCk5wqrkjJVKHFqlrRWkGY6t6eOTFfw893mXUu8Tr9ExnHLDjQMGCXCXBnLlKahK+ZQKgt7HgT6yrtr6+6pYe6As0ZTckFVUtLTrXeqCSxUcbl8yvZW3e0hpcnGDTyU+41P2cEvtqe2GNGeEnuWuIaE7TIrsQvfIRDs19GzAAMIfo+v8ovOjEAHJY5I4Y

## 코드 정리 후 DF 구성

In [ ]:
# 20번 스크롤해서 찾은 모든 상품에 대한 상품명, 가격, 링크 추출(출력)
name, price, link = [[] for _ in range(3)]

for item in lis:
    try:
        n = item.find_element(By.CSS_SELECTOR,'.basicProductCardInformation_title__Bc_Ng').text    
        p = item.find_element(By.CSS_SELECTOR,'.priceTag_inner_price__TctbK').text.split('\n')[0].replace(',','')
        l = item.find_element(By.TAG_NAME,'a').get_attribute('href')
    except:
        n = '판매중단'
        p = '판매중단'
        l = '판매중단'
    name.append(n)
    price.append(p)
    link.append(l)

In [ ]:
df = pd.DataFrame({"품명":name, "가격":price, "link":link})

In [6]:
browser.close()
browser.switch_to.window(browser.window_handles[0])
browser.close()

### 창의 크기가 작을 때
- 모바일 버전으로 변경된 것
    - 태그를 다시 확인해서 진행할 필요가 생긴다